# MNIST Dataset
The following `Jupyter Notebook` has been *adapted* from the [Keras blog article](https://blog.keras.io/building-autoencoders-in-keras.html) written by *F. Chollet* on [autoencoders](https://en.wikipedia.org/wiki/Autoencoder), and will focus on training the `MinimalAutoencoder` (aka *vanilla autoencoder*) on the `MNIST` dataset.

## Setup
Need to get the necessary packages ...

In [ ]:
# check for colab
if "google.colab" in str(get_ipython()):
  # install colab dependencies
  !pip install git+https://github.com/DiogenesAnalytics/autoencoder

## Get MNIST Data
Wille use `keras.datasets` to get the `MNIST` dataset, and then do some *normalizing* and *reshaping* to prepare it for the *autoencoder*.

In [ ]:
# get necessary libs for data/preprocessing
import tensorflow as tf
from keras.datasets import mnist

# load the data
(x_train, _), (x_test, _) = mnist.load_data()

# preprocess the data (normalize)
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

# convert to tf datasets
train_ds = tf.data.Dataset.from_tensor_slices((x_train, x_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, x_test))

# set a few params
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

# update with batch/buffer size
train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

## Autoencoder Training
Finally the *autoencoder* can be trained ...

In [ ]:
# get libs for training ae
from autoencoder.model.minimal import Min2DAE, Min2DParams

# seupt config
config = Min2DParams(
    l0={"input_shape": (28, 28)},
    l2={"units": 32 * 1},
    l3={"units": 28 * 28 * 1},
    l4={"target_shape": (28, 28)},
)

# get ae instance
autoencoder = Min2DAE(config)

# check network topology
autoencoder.summary()

In [ ]:
# get early stopping class
from keras.callbacks import EarlyStopping

# create callback
early_stop_callback = EarlyStopping(monitor="val_loss", patience=3)

# compile ae
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")

# begin model fit
autoencoder.fit(
    x=train_ds,
    epochs=50,
    validation_data=test_ds,
    callbacks=[early_stop_callback],
)

In [ ]:
# view training loss
autoencoder.training_history()

## Visualizing Predictions
Now it is possible, using the trained autoencoder, to encode/decode an image and see how it compares to the original ...

In [ ]:
# get viz func
from autoencoder.data import compare_image_predictions

# get samples from validation dataset
val_samples = test_ds.take(1)

# get raw numpy arrays
val_input = [item for pair in val_samples.as_numpy_iterator() for item in pair[0]]

# and decoded
decoded_imgs = autoencoder.predict(x=val_samples)

# display
compare_image_predictions(val_input, decoded_imgs)

## Reconstruction Error Distribution
Now let us take peak into this dataset and see how well the *autoencoder* is capturing the image features.

In [ ]:
# get custom class
from autoencoder.data.evaluate import AutoencoderEvaluator

# get instance
ae_eval = AutoencoderEvaluator(autoencoder, test_ds, axis=(1, 2))

# view distribution
ae_eval.view_error_distribution("MNIST Autoencoder: Reconstruction Error Distribution")